<a href="https://colab.research.google.com/github/hrishikesav/BigData/blob/master/Bank_CreditCardDefaulterList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import files
uploaded = files.upload()

Saving loan.txt to loan.txt


In [16]:
#!{'mkdir -p data'}
!ls

cards.txt  data  loan.txt  sample_data


In [5]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

>> pip install --quiet apache-beam



In [19]:
import apache_beam as beam

def calculate_points(element):
  cust_id,first_name,last_name, relationship_id,card_type,max_limit,total_spent,cash_withdrawn,payment_cleared,payment_date = element.split(",")

  total_spent = int(total_spent)
  cash_withdrawn = int(cash_withdrawn)
  payment_cleared = int(payment_cleared)
  max_limit = int(max_limit)

  key_name = cust_id + "," + first_name + " " + last_name

  defaulter_points = 0

  if(payment_cleared < (total_spent * 0.7)):
    defaulter_points += 1

  if(total_spent >= max_limit) and (payment_cleared < total_spent):
    defaulter_points += 1

  if(total_spent >= max_limit) and (payment_cleared < (total_spent * 0.7)):
    defaulter_points += 1

  return key_name,defaulter_points

p = beam.Pipeline()

card_defaulter = (
    p
    | "Read card data" >> beam.io.ReadFromText('cards.txt',skip_header_lines=1)
    | "Process card data" >> beam.Map(calculate_points)
    | "Add points" >> beam.CombinePerKey(sum)
    | "Minimum one default point" >> beam.Filter(lambda element: element[1] > 0)
    | "Write ouput" >> beam.io.WriteToText('data/card-defaulter')
)

p.run()

!{('head -n 20 data/card-defaulter-00000-of-00001')}

('CT28383,Miyako Burns', 3)
('CT74474,Nanaho Brennan', 3)
('CT66322,Chris Bruce', 1)
('CT65528,Bonnie Barlow', 2)
('CT84463,Isaac Bowman', 4)
('CT12838,Isidore Albert', 5)
('CT96185,Danielle Bradshaw', 3)
('CT74827,Hanna Boyer', 3)
('CT98239,Sayuri Case', 4)
('CT57141,Kaori Brown', 2)
('CT85915,Dean Becker', 5)
('CT86894,Miwa Carter', 5)
('CT33583,Odette Case', 4)
('CT64751,Youko Bryant', 3)
('CT14528,Omar Acevedo', 6)
('CT98422,Nicole Caldwell', 2)
('CT66224,Ivan Blanchard', 2)
('CT91012,Jerry Barron', 4)
('CT64089,Fay Beard', 4)
('CT56854,Michael Cannon', 4)
